## Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import SGDClassifier

from jenga.basis import Dataset
from jenga.models.model import Model
from jenga.corruptions.perturbations import Perturbation
from jenga.cleaning.imputation import MeanModeImputation, DatawigImputation
from jenga.cleaning.outlier_detection import PyODKNN, PyODIsolationForest

In [2]:
seed = 10

In [3]:
dataset = Dataset(seed, "credit-g")

Dataset 'credit-g', target: 'class'
**Author**: Dr. Hans Hofmann  

**Source**: [UCI](https://archive.ics.uci.edu/ml/datasets/statlog+(german+credit+data)) - 1994    

**Please cite**: [UCI](https://archive.ics.uci.edu/ml/citation_policy.html)



**German Credit data**  

This dataset classifies people described by a set of attributes as good or bad credit risks.



This dataset comes with a cost matrix: 

``` 

      Good  Bad (predicted)  

Good   0    1   (actual)  

Bad    5    0  

```



It is worse to class a customer as go

Attribute types: 


,attribute_names,categorical_indicator
0,checking_status,True
1,duration,False
2,credit_history,True
3,purpose,True
4,credit_amount,False
5,savings_status,True
6,employment,True
7,installment_commitment,False
8,personal_status,True
9,other_parties,True


In [4]:
all_data = dataset.all_data
all_data

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,...,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes,good
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,...,real estate,22.0,none,own,1.0,skilled,1.0,none,yes,bad
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,...,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes,good
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,...,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes,good
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,...,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes,bad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no checking,12.0,existing paid,furniture/equipment,1736.0,<100,4<=X<7,3.0,female div/dep/mar,none,...,real estate,31.0,none,own,1.0,unskilled resident,1.0,none,yes,good
996,<0,30.0,existing paid,used car,3857.0,<100,1<=X<4,4.0,male div/sep,none,...,life insurance,40.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes,good
997,no checking,12.0,existing paid,radio/tv,804.0,<100,>=7,4.0,male single,none,...,car,38.0,none,own,1.0,skilled,1.0,none,yes,good
998,<0,45.0,existing paid,radio/tv,1845.0,<100,1<=X<4,4.0,male single,none,...,no known property,23.0,none,for free,1.0,skilled,1.0,yes,yes,bad


In [5]:
attribute_names = dataset.attribute_names
attribute_names

['checking_status',
 'duration',
 'credit_history',
 'purpose',
 'credit_amount',
 'savings_status',
 'employment',
 'installment_commitment',
 'personal_status',
 'other_parties',
 'residence_since',
 'property_magnitude',
 'age',
 'other_payment_plans',
 'housing',
 'existing_credits',
 'job',
 'num_dependents',
 'own_telephone',
 'foreign_worker']

In [6]:
attribute_types = dataset.attribute_types
attribute_types

,attribute_names,categorical_indicator
0,checking_status,True
1,duration,False
2,credit_history,True
3,purpose,True
4,credit_amount,False
5,savings_status,True
6,employment,True
7,installment_commitment,False
8,personal_status,True
9,other_parties,True


In [7]:
categorical_columns = dataset.categorical_columns
categorical_columns

['checking_status',
 'credit_history',
 'purpose',
 'savings_status',
 'employment',
 'personal_status',
 'other_parties',
 'property_magnitude',
 'other_payment_plans',
 'housing',
 'job',
 'own_telephone',
 'foreign_worker']

In [8]:
numerical_columns = dataset.numerical_columns
numerical_columns

['duration',
 'credit_amount',
 'installment_commitment',
 'residence_since',
 'age',
 'existing_credits',
 'num_dependents']

### Visualize the dataset

In [ ]:
## plot the original dataset
def hide_current_axis(*args, **kwds):
        plt.gca().set_visible(False)
        
def plot_data(data):
    sns.set_style("white") # grid/no grid style: darkgrid, whitegrid, dark, white, ticks
    
    plot = sns.pairplot(data, hue="class")
    plot.map_upper(hide_current_axis)
    plt.show()

In [ ]:
plot_data(all_data)

### Get training and test sets

In [9]:
train_data, train_labels, test_data, test_labels = dataset.get_train_test_data()

display(train_data.head())
print(train_labels[0:5])

display(test_data.head())
print(test_labels[0:5])

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
188,<0,12.0,existing paid,radio/tv,674.0,100<=X<500,4<=X<7,4.0,male mar/wid,none,1.0,life insurance,20.0,none,own,1.0,skilled,1.0,none,yes
194,0<=X<200,45.0,existing paid,radio/tv,3031.0,100<=X<500,1<=X<4,4.0,male single,guarantor,4.0,life insurance,21.0,none,rent,1.0,skilled,1.0,none,yes
225,no checking,36.0,no credits/all paid,repairs,2613.0,<100,1<=X<4,4.0,male single,none,2.0,car,27.0,none,own,2.0,skilled,1.0,none,yes
580,0<=X<200,18.0,critical/other existing credit,new car,1056.0,<100,>=7,3.0,male single,guarantor,3.0,real estate,30.0,bank,own,2.0,skilled,1.0,none,yes
428,no checking,9.0,existing paid,furniture/equipment,1313.0,<100,>=7,1.0,male single,none,4.0,car,20.0,none,own,1.0,skilled,1.0,none,yes


['bad' 'bad' 'good' 'bad' 'good']


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.0,delayed previously,used car,2993.0,<100,1<=X<4,3.0,male single,none,2.0,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.0,critical/other existing credit,radio/tv,3656.0,no known savings,>=7,4.0,male single,none,4.0,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,12.0,critical/other existing credit,new car,1255.0,<100,>=7,4.0,male single,none,4.0,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,8.0,existing paid,radio/tv,1414.0,<100,1<=X<4,4.0,male single,guarantor,2.0,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.0,critical/other existing credit,new car,691.0,<100,>=7,4.0,male single,none,3.0,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes


['good' 'good' 'good' 'good' 'bad']


## Model

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


# model parameters
learner = SGDClassifier(max_iter=1000)
param_grid = {
    'learner__loss': ['log'],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}


# preprocessing pipeline for numerical columns
transformer_numeric = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('standard_scale', StandardScaler())
])

# preprocessing pipeline for categorical columns
transformer_categorical = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='__NA__')),
    ('one_hot_encode', OneHotEncoder(handle_unknown='ignore'))
])

# preprocessor
feature_transform = ColumnTransformer(transformers=[
    ('categorical_features', transformer_categorical, categorical_columns),
    ('numerical_features', transformer_numeric, numerical_columms)
])

## prediction pipeline: append classifier (learner) to the preprocessing pipeline
pipeline = Pipeline([
    ('features', feature_transform),
    ('learner', learner)
])

In [ ]:
model_obj = Model(seed, train_data, train_labels, test_data, test_labels, categorical_columns, numerical_columms, pipeline, learner, param_grid)

model = model_obj.fit_model(train_data, train_labels)

## Corruptions

In [10]:
# corruption perturbations to apply
corr_perturbations = Perturbation(categorical_columns, numerical_columns)

In [11]:
test_data_corrupted, perturbations, cols_perturbed = corr_perturbations.apply_perturbation(test_data, 5)
test_data_corrupted.head(10)

Applying perturbations...
GaussianNoise: {'column': 'residence_since', 'fraction': 0.25} on column ['residence_since']
MissingValues: {'column': 'credit_history', 'fraction': 0.5, 'na_value': nan, 'missingness': 'MAR'} on column ['credit_history']
GaussianNoise: {'column': 'duration', 'fraction': 0.25} on column ['duration']
MissingValues: {'column': 'credit_amount', 'fraction': 0.75, 'na_value': nan, 'missingness': 'MAR'} on column ['credit_amount']
SwappedValues: {'column_a': 'housing', 'column_b': 'personal_status', 'fraction': 0.25} on column ['housing', 'personal_status']


,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.000000,delayed previously,used car,NaN,<100,1<=X<4,3.0,male single,none,2.000000,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.000000,critical/other existing credit,radio/tv,NaN,no known savings,>=7,4.0,male single,none,4.000000,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,8.804100,critical/other existing credit,new car,NaN,<100,>=7,4.0,male single,none,4.000000,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,8.000000,NaN,radio/tv,NaN,<100,1<=X<4,4.0,male single,guarantor,-2.096262,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.000000,NaN,new car,NaN,<100,>=7,4.0,male single,none,3.000000,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes
955,<0,24.000000,NaN,radio/tv,NaN,>=1000,>=7,4.0,female div/dep/mar,none,4.000000,life insurance,57.0,none,rent,2.0,high qualif/self emp/mgmt,1.0,yes,yes
121,no checking,24.000000,NaN,used car,NaN,<100,>=7,4.0,rent,none,2.000000,car,41.0,none,female div/dep/mar,2.0,high qualif/self emp/mgmt,1.0,yes,yes
230,>=200,23.139422,NaN,radio/tv,NaN,<100,1<=X<4,4.0,male single,none,2.000000,car,26.0,none,own,1.0,skilled,1.0,none,yes
11,<0,48.000000,NaN,business,4308.0,<100,<1,3.0,female div/dep/mar,none,4.000000,life insurance,24.0,none,rent,1.0,skilled,1.0,none,yes
120,<0,-9.360327,NaN,radio/tv,1835.0,<100,1<=X<4,3.0,female div/dep/mar,none,2.000000,real estate,25.0,none,own,2.0,skilled,1.0,yes,yes


In [12]:
perturbations

In [13]:
cols_perturbed

['residence_since',
 'credit_history',
 'duration',
 'credit_amount',
 'housing',
 'personal_status']

### Visualize the original and corrupted test set

In [ ]:
## original test data
plot_data(pd.concat([test_data, pd.Series(test_labels, name='class')], axis=1))

In [ ]:
## corrupted test data
plot_data(pd.concat([test_data_corrupted, pd.Series(test_labels, name='class')], axis=1))

## Cleaning

### Imputation

In [ ]:
mean_mode_imputer = MeanModeImputation(train_data, test_data_corrupted, categorical_columns, numerical_columns)

test_data_mm_imputed = mean_mode_imputer.fit_transform(train_data, test_data_corrupted)
test_data_mm_imputed

In [ ]:
datawig_imputer = DatawigImputation(train_data, test_data_corrupted, categorical_columns, numerical_columms)

test_data_dw_imputed = datawig_imputer.fit_transform(train_data, test_data_corrupted)
test_data_dw_imputed

##### Using PPP

In [ ]:
# for all imputers return scores, take best
# using ppp

In [ ]:
from jenga.cleaning.imputation import MeanModeImputation, DatawigImputation
from jenga.cleaning.ppp import PipelinePerformancePrediction

In [ ]:
learner = SGDClassifier(max_iter=1000)
param_grid = {
    'learner__loss': ['log'],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}

In [ ]:
ppp = PipelinePerformancePrediction(seed, dataset, categorical_columns, numerical_columns, learner, param_grid)

In [ ]:
### FIX, take this out of ppp
train_data = ppp.train_data
train_labels = ppp.train_labels

test_data = ppp.test_data
test_labels = ppp.test_labels

In [ ]:
# generate corrpted test data
test_data_corrupted, perturbations, cols_perturbed = ppp.get_corrupted(test_data)

In [ ]:
imputer_candidates = [MeanModeImputation, DatawigImputation]

In [ ]:
imputers = []
for imputer in imputer_candidates:
    imputers.append(imputer(train_data, test_data_corrupted, categorical_columns, numerical_columns))

In [ ]:
imputers

In [ ]:
ppp_model = ppp.fit_ppp(train_data)

In [ ]:
score_no_cleaning = ppp.predict_score_ppp(ppp_model, test_data)
score_no_cleaning

In [ ]:
imputed_scores_ppp = []
for imputer in imputers:
    test_data_imputed = imputer.fit_transform(train_data, test_data_corrupted)
    imputed_score = ppp.predict_score_ppp(ppp_model, test_data_imputed)
    print(f"PPP score with {imputer}: {imputed_score}")
    imputed_scores_ppp.append(imputed_score)

In [ ]:
imputed_scores_ppp

##### Using PPP and Cleaner classes

In [28]:
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNN, PyODIsolationForest
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, DatawigImputation
from jenga.cleaning.cleaner import Cleaner

In [29]:
cleaner_candidates = [
    (NoOutlierDetection, NoImputation),
    (NoOutlierDetection, MeanModeImputation),
    (NoOutlierDetection, DatawigImputation),
    (PyODKNN, NoImputation),
    (PyODKNN, MeanModeImputation),
    (PyODKNN, DatawigImputation),
    (PyODIsolationForest, NoImputation),
    (PyODIsolationForest, MeanModeImputation),
    (PyODIsolationForest, DatawigImputation)
]

In [30]:
cleaners = []
for outd, imp in cleaner_candidates:
    cleaners.append(Cleaner(train_data, 
                            test_data_corrupted, 
                            categorical_columns, 
                            numerical_columns, 
                            outlier_detection = outd(train_data, 
                                                     test_data_corrupted, 
                                                     categorical_columns, 
                                                     numerical_columns), 
                            imputation = imp(train_data, 
                                             test_data_corrupted, 
                                             categorical_columns, 
                                             numerical_columns)
                           ))

In [31]:
learner = SGDClassifier(max_iter=1000)
param_grid = {
    'learner__loss': ['log'],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
}

In [32]:
ppp = PipelinePerformancePrediction(seed, dataset, categorical_columns, numerical_columns, learner, param_grid)

In [33]:
ppp_model = ppp.fit_ppp(train_data)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    7.0s finished


In [36]:
score_no_cleaning = ppp.predict_score_ppp(ppp_model, test_data)
score_no_cleaning

0.4844074844074844

In [17]:
from jenga.cleaning.outlier_detection import NoOutlierDetection
from jenga.cleaning.imputation import NoImputation


class Cleaner:
    
    def __init__(self, 
                 df_train,
                 df_corrupted,
                 categorical_columns,
                 numerical_columns,
                 outlier_detection=NoOutlierDetection, 
                 imputation=NoImputation):
        self.outlier_detection = outlier_detection
        self.imputation = imputation
        
    
    def apply_cleaner(self, df_train, df_corrupted, categorical_columns, numerical_columns):
        df_cleaned = self.outlier_detection(df_train, df_corrupted, 
                                            categorical_columns, 
                                            numerical_columns).fit_transform(df_train, df_corrupted)
        
        # do something for fixing/removing the outliers
        if 'outlier' in df_cleaned.columns:
            ### TODO 
            df_cleaned = df_cleaned.drop('outlier', axis=1)
            
        # impute
        df_cleaned = self.imputation(df_train, df_cleaned, 
                                     categorical_columns, 
                                     numerical_columns).fit_transform(df_train, df_corrupted)
        
        return df_cleaned

In [18]:
cleaner = Cleaner(train_data, test_data_corrupted, categorical_columns, numerical_columns)

In [20]:
df_cleaned = cleaner.apply_cleaner(train_data, test_data_corrupted, categorical_columns, numerical_columns)
df_cleaned

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
841,no checking,21.000000,delayed previously,used car,NaN,<100,1<=X<4,3.0,male single,none,2.000000,real estate,28.0,stores,own,2.0,unskilled resident,1.0,none,yes
956,>=200,30.000000,critical/other existing credit,radio/tv,NaN,no known savings,>=7,4.0,male single,none,4.000000,life insurance,49.0,stores,own,2.0,unskilled resident,1.0,none,yes
544,no checking,8.804100,critical/other existing credit,new car,NaN,<100,>=7,4.0,male single,none,4.000000,real estate,61.0,none,own,2.0,unskilled resident,1.0,none,yes
173,0<=X<200,8.000000,NaN,radio/tv,NaN,<100,1<=X<4,4.0,male single,guarantor,-2.096262,real estate,33.0,none,own,1.0,skilled,1.0,none,no
759,<0,12.000000,NaN,new car,NaN,<100,>=7,4.0,male single,none,3.000000,life insurance,35.0,none,own,2.0,skilled,1.0,none,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,<0,1.414018,existing paid,repairs,11998.0,<100,<1,1.0,male div/sep,none,1.000000,no known property,34.0,none,own,1.0,unskilled resident,1.0,yes,yes
192,0<=X<200,55.133147,existing paid,business,3915.0,<100,1<=X<4,4.0,male single,none,2.000000,car,36.0,none,own,1.0,skilled,2.0,yes,yes
398,0<=X<200,12.000000,existing paid,new car,NaN,<100,>=7,1.0,rent,none,1.000000,real estate,46.0,none,male div/sep,2.0,skilled,1.0,none,yes
450,no checking,36.000000,NaN,used car,NaN,no known savings,1<=X<4,4.0,male single,none,2.000000,car,30.0,none,own,1.0,high qualif/self emp/mgmt,1.0,yes,yes


In [ ]:
from jenga.cleaning.ppp import PipelinePerformancePrediction

from jenga.cleaning.imputation import MeanModeImputation, DatawigImputation


class Clean:
    
    def __init__(self, )

### Outlier Detection

In [ ]:
# detection using KNN from PyOD
outlier = PyODKNN(train_data, test_data_corrupted, categorical_columns, numerical_columms)

In [ ]:
test_data_corrupted_outliers = outlier.fit_transform(train_data, test_data_corrupted)
test_data_corrupted_outliers.head(10)

In [ ]:
# detection using Isolation Forest from PyOD
outlier_if = PyODIsolationForest(train_data, test_data_corrupted, categorical_columns, numerical_columms)

In [ ]:
test_data_corrupted_outliers_if = outlier_if.fit_transform(train_data, test_data_corrupted)
test_data_corrupted_outliers_if.head(10)

#### Preparing the outliers for imputation

In [ ]:
if "outlier" in test_data_corrupted_outliers.columns:
    print(f'Setting {test_data_corrupted_outliers["outlier"].sum()} to Nan')
    test_data_corrupted_outliers.loc[test_data_corrupted_outliers["outlier"], :] = np.nan
    test_data_corrupted_outliers = test_data_corrupted_outliers.drop('outlier', axis=1)

In [ ]:
## train_data, test_data_corrupted, 
## check values in column in the training data -> check for outliers in the same column in the corrupted data
## store .loc 
## convert those .loc for those column into nan
## impute

In [ ]:
numerical_columms

In [ ]:
test_data_corrupted

## Evaluation

In [ ]:
# score without cleaning
model_obj.score_on_test_data(model.predict_proba(test_data))

In [ ]:
# score with corruptions
model_obj.score_on_test_data(model.predict_proba(test_data_corrupted))

In [ ]:
# score with mean/mode imputation
model_obj.score_on_test_data(model.predict_proba(test_data_mm_imputed))

In [ ]:
# score with datawig imputation
model_obj.score_on_test_data(model.predict_proba(test_data_dw_imputed))